In [1]:
import pandas as pd
import numpy as np

import scipy.io as sio
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['font.size'] = 8.0
matplotlib.rcParams['pdf.fonttype'] = 42

In [2]:
def getTrialAlignment(index, unittp):
    repeats = []
    for idx_st, idx_stop in zip(index['index_starts'], index['index_stops']):
        rep = unittp[(unittp > idx_st) & (unittp < idx_stop)].values
        rep = rep - idx_st
        repeats.append(rep)
    return repeats

In [3]:
# input all parameters
Fs = 3e4
TimeHistory = 20e-3 # in seconds
TimePost = 2e-3 # in seconds

In [4]:
# read in the index based on stimulus repeats
name = 'm10_whiteNoise_2'
index = pd.read_csv('./dataFolder/' + name + '_index.csv')
# convert index to timestamp
index_tp = index * 1/Fs

In [5]:
# read in the whiteNoise stimulus
data = pd.read_csv('./dataFolder/' + name + '_RawData.csv')
Stim = data['stimulus']

# get the first WN repeat 
start = index.loc[0, :][0]
stop = index.loc[0, :][1]
WhiteNoise = Stim[start:stop]
# reindex the WhiteNoise to reflect in seconds
WhiteNoise.index = np.arange(0, len(WhiteNoise))*1/Fs

In [6]:
# get spike timestamp
filename = r'G:\Other computers\My Computer\TanviStuff\PiliferRecordings\2022_02_04\m10_whiteNoise_2.txt'
T = pd.read_csv(filename, sep = '\t')
tp = T.loc[:,'Timestamp']

#loop through all units
NumUnits = np.unique(T['Unit'])
all_Units = {}
for idx, unit in enumerate(NumUnits):
    all_Units['unit_' + str(idx)] = T.loc[T['Unit'] == unit, 'Timestamp']
    
# align the spikes to stimulus repeats
all_Units_aligned = {}
for k in all_Units.keys():
    repeats = getTrialAlignment(index_tp, all_Units[k])
    all_Units_aligned[k] = repeats

In [7]:
# get prior timestamp

import random

num_repeats = len(index_tp)
num_permuts = 500
all_Units_prior = {}
for k in all_Units_aligned.keys():
    ## decide how many repeats to we want to compute for the prior 
    ## and also how many spikes in each repeat
#     l = []
#     for eachRep in all_Units_aligned[k]:
#         l.append(len(eachRep))
#     num_permuts = int(np.mean(l))  
    randomlist = [ [ None for y in range(num_permuts)]
             for x in range(num_repeats)]
    for jj in range(num_repeats):
        for ii in range(num_permuts):
            n = random.randint(20*1e-3*Fs,len(WhiteNoise))*1/Fs
            randomlist[jj][ii] = n
    all_Units_prior[k] = randomlist

In [8]:
# pickle dump the timestamps
import pickle

saveObject = (all_Units_aligned,all_Units_prior, WhiteNoise)
with open('./dataFolder/' + name + '_timeStamps.pickle',"wb") as f:
    pickle.dump(saveObject, f)